In [1]:
using JuMP
using Ipopt
using Complementarity

In [38]:
GU = Dict(:pi => 0.01, :L=> 0.5, :sigma => 2, :kother=>.5)
GU[:rho] = 1 - 1/GU[:sigma]

0.5

In [87]:
function nash(GU)
    
    pi = GU[:pi]
    L = GU[:L]
    σ = GU[:sigma]
    kother = GU[:kother]
    ρ = GU[:rho]    

    m = Model(Ipopt.Optimizer)
    @variables(m,begin
        C_G>=1e-5, (start = 1,)
        C_B>=1e-5, (start = 1,)
        Gamma>=0
        K, (start = 1,)
        EU>=(1-pi)*1^ρ/ρ + pi* *(1-L)^ρ/ρ
    end)

    @NLobjective(m,Max, (Gamma-pi)*(K-kother))

    @NLconstraints(m,begin
        eudef, EU == (1-pi)*C_G^ρ/ρ + pi * C_B^ρ/ρ
        budget_G, C_G == 1-Gamma*K
        budget_B, C_B == 1-L + (1-Gamma)*K
        coverage, Gamma*((1-pi)*C_G^(ρ-1) + pi*C_B^(ρ-1)) >= pi*C_B^(ρ-1)
    end)


    return m
end

nash (generic function with 1 method)

In [88]:
GU[:kother] = 0

log = Dict()
for n∈1:5
    dev = 1
    #GU[:kother] = 
    for iter∈1:25
        if dev ≈ 0
            break
        end
        Nash = nash(GU)
        set_silent(Nash)
        optimize!(Nash)

        log[n,iter] = Dict(:dev => dev,
            :K => value(Nash[:K]),
            :Gamma => value(Nash[:Gamma]),
            :Profit => objective_value(Nash),
            :C_G => value(Nash[:C_G]),
            :C_B => value(Nash[:C_B])
        )

        dev = abs(GU[:kother] - value(Nash[:K])*((n-1)/n))
        GU[:kother] = value(Nash[:K])*((n-1)/n)
    end
end

In [91]:
log[5,9]

Dict{Symbol, Float64} with 6 entries:
  :Gamma  => 0.0104315
  :C_B    => 1.45087
  :K      => 0.960892
  :Profit => 8.29436e-5
  :dev    => 0.000131072
  :C_G    => 0.989976